### 使用 pattern mining找出所有的pattern，然后按照最常用的组合进行推荐
按照学生人数为110计算 实际学生为134，但是有些学生的log不符合要求。

In [9]:
from prefixspan import PrefixSpan

db = [
    [0, 1, 2, 3, 4],
    [1, 1, 1, 3, 4],
    [2, 1, 2, 2, 0],
    [1, 1, 1, 2, 2],
]

ps = PrefixSpan(db)

In [12]:
print(ps.frequent(2))

[(2, [0]), (4, [1]), (3, [1, 2]), (2, [1, 2, 2]), (2, [1, 3]), (2, [1, 3, 4]), (2, [1, 4]), (2, [1, 1]), (2, [1, 1, 1]), (3, [2]), (2, [2, 2]), (2, [3]), (2, [3, 4]), (2, [4])]


In [45]:
import re
import json
import pandas as pd
# import datetime
import re
from collections import Counter
import numpy as np
import time
import math
import sys
quiz = pd.read_csv(filepath_or_buffer= "C:\\Users\\zhazhang\\Desktop\\GitHub\\quizzResult.csv")
data1 = pd.read_csv(filepath_or_buffer="C:\\Users\\zhazhang\\Desktop\\GitHub\\event_semester1.csv")
exam = pd.read_csv(filepath_or_buffer="C:\\Users\\zhazhang\\Desktop\\GitHub\\exam_result.csv")
data1 = (data1[data1['action'].isin(['Viewed'])])
quiz.score = quiz.score.map(lambda x: x*10)
exam['score'] = exam.score.map(lambda x: (x/18*100))
quiz['action'] = "Submitted"
event = pd.concat([data1[['userId','objectId','action','Timestamp']],quiz[['userId','objectId','action','Timestamp']]])
exam['action']='FinalExam'
exam['Timestamp'] = 1542835584754#1542835584754 # this is the right timestamp of exam
exam['objectId'] = 11209 # I faked an
event = pd.concat([event, exam[['userId','objectId','action','Timestamp']]])
event = event[event.Timestamp < 1542835584755] # here cut the data at the time just after exam.
"""
Here is the treat of event which add the correlated quiz and exam score 
"""
user_list = list(set(event['userId']))
next_score = []
quiz1 = list(set(quiz.objectId))[0]
quiz2 = list(set(quiz.objectId))[1]
exam_time = exam.iloc[0]['Timestamp']
user_quized = []
# print(len(user_list))
for i in user_list:
    if ((len(quiz[quiz.userId==i])) == 2) & (i in list(exam.userId)):
        #quiz=quiz[quiz.userId!=i]
        user_quized.append(i)
#print(len(user_list))
event = event[event.userId.isin(user_quized)]
#print(len(event))
for i in event.itertuples():
#     print(i)
    user = i.userId
    timestamp = i.Timestamp
    quiz1_time = event[(event.userId == user) & (event.objectId == quiz1)].Timestamp
    quiz2_time = event[(event.userId == user) & (event.objectId == quiz2)].Timestamp
    slot_1 = (-(timestamp-quiz1_time)).iloc[0]
    slot_2 = (-(timestamp-quiz2_time)).iloc[0]
    slot_e = -(timestamp-exam_time)
    l = [slot_1, slot_2, slot_e] 
    min0 = 10000000000

    for j in range(len(l)):
        if (l[j] >= 0) & (l[j]<min0):
            min0 = l[j]

    if min0 == l[0]:
        next_score.append(quiz[(quiz.userId==user)&(quiz.objectId==quiz1)].score.iloc[0])
    elif min0 == l[1]:
        next_score.append(quiz[(quiz.userId==user)&(quiz.objectId==quiz2)].score.iloc[0])
    elif min0 == l[2]:
        next_score.append(exam[(exam.userId==user)].score.iloc[0])
    else:
        next_score.append(exam[(exam.userId==user)].score.iloc[0])

event["nearest_score"] = next_score
event=event.sort_values(by='Timestamp', ascending=True)
resourceIndex = {8935: 0, 8936: 1, 316752: 2, 321242: 3, 321240: 4, 321237:5, 11209:6 }
l = list(resourceIndex.keys())
event1 = event[event["objectId"] == int(l[0])]
#or event["objectId"] == int(l[1]) or event["objectId"] == int(l[2]) or event["objectId"] == int(l[3]) or event["objectId"] == int(l[4]) or event["objectId"] == int(l[5]) or event["objectId"] == int(l[6])or event["objectId"] == int(l[7])or event["objectId"] == int(l[8]) or event["objectId"] == int(l[9])]
event2 = event[event["objectId"] == int(l[1])]
event3 = event[event["objectId"] == int(l[2])]
event4 = event[event["objectId"] == int(l[3])]
event5 = event[event["objectId"] == int(l[4])]
event6 = event[event["objectId"] == int(l[5])]
event7 = event[event["objectId"] == int(l[6])]
# event8 = event[event["objectId"] == int(l[7])]
# event9 = event[event["objectId"] == int(l[8])]
# event10 = event[event["objectId"] == int(l[9])]
event = pd.concat([event1,event2,event3,event4,event5,event6,event7])#,event8,event9,event10

In [46]:
exam=exam.sort_values(by='score', ascending=False)

Here is the traning dataset. We will training the pattern with this part 

In [51]:
s_good = [] #top 30%
s_normal = []# middle 40%
s_bad = []# next 30%
for i in range(len(exam)):
    if i < 30:
        s_good.append(exam.iloc[i]["userId"])
    if i>=30 & i<70:
        s_normal.append(exam.iloc[i]["userId"])
    elif i>=70 & i<100:
        s_bad.append(exam.iloc[i]["userId"])
        
print("student level dividing done")

def get_key(dic, value):
    """
    know the value, get keys
    """
    a = [k for k, v in dic.items() if v == value]
    return a[0]

student level dividing done


Training dataset organisation

In [56]:
l_good = []
l_bad = []
l_normal = []
for user in s_good:
    l_path = []
    for i in range(len(event)):
        if user==event.iloc[i]["userId"]:
            l_path.append(event.iloc[i]["objectId"])
    l_good.append(l_path)
for user in s_normal:
    l_path = []
    for i in range(len(event)):
        if user==event.iloc[i]["userId"]:
            l_path.append(event.iloc[i]["objectId"])
    l_normal.append(l_path)
for user in s_bad:
    l_path = []
    for i in range(len(event)):
        if user==event.iloc[i]["userId"]:
            l_path.append(event.iloc[i]["objectId"])
    l_bad.append(l_path)

### recommendation def

In [186]:
resourcelist = list(set(event.objectId))
def recommend(trainingset=l_normal, s_group=s_normal, student=s_normal[0], path_length=5, rl=resourcelist):
    
    # Here we put the influence or this stdent's learning log bigger. x10
    for i in range(10):
        trainingset.append(trainingset[s_group.index(student)])
    ps = PrefixSpan(trainingset)

    pattern = ps.topk(50, filter=lambda patt, matches: len(patt)>1)# pattern lenth should bigger than 1
    pattern_time = {} #Here stores all pattern with appear times
    
    for i,element in enumerate(pattern):
        l_s = []# store pattern in this element
        s = ""
        for i in range(len(element[1])):
            if i==0:
                s=str(element[1][i])
            else:
                l_s.append(s+","+str(element[1][i]))
                s = str(element[1][i])
        for j in l_s:
            if j in pattern_time.keys():
                pattern_time[j]+=element[0]
            else:
                pattern_time[j]=element[0]
                
    # ordered pattern in list            
    pattern_time = sorted(pattern_time.items(),key = lambda pattern_time:pattern_time[1],reverse=True)
    print("pattern with time:",pattern_time)
    # delete repeat part
    #print(len(pattern_time))
    
    """
    Here is deduplication.
    we can't delete the item of list in for cycle. It will have 'index out of range problem'. 
    So we store the repeat index and delete after
    """ 
    delete_indice = [] 
    for k1 in range(len(pattern_time)):
        starter = pattern_time[k1][0].split(",")[0]
        ender = pattern_time[k1][0].split(",")[1]
        if starter == ender:
            delete_indice.append(k1)
        if pattern_time[k1]==pattern_time[-1]:
            break
            
        for k2 in range(k1+1,len(pattern_time)):
            #print(pattern_time[k2])
            temps_start = pattern_time[k2][0].split(",")[0]
            temps_end = pattern_time[k2][0].split(",")[1]
            if starter == temps_start:
                delete_indice.append(pattern_time[k2])
            if ender == pattern_time[k2][0].split(",")[1]:
                delete_indice.append(pattern_time[k2])    
                
    for  i in set(delete_indice):
        if i in pattern_time:
            pattern_time.remove(i)
        
    """
    Here we organise the path from pattern list.
    We should firstly find the head then finish the path.
    """
    element = []
       
    pattern_result = [x[0] for x in pattern_time] # delete pattern show times, keep pattern
    print("unique pattern:",pattern_result)  
    store = []
    for i in range(len(pattern_result)):
        for j in range(len(pattern_result)):
            if i==j:
                continue
            if pattern_result[i].split(",")[0] in pattern_result[j]:
                store.append(pattern_result[i])
    path =list(set(pattern_result).difference(set(store)))[0]
    print("begin_node of path:",path)
    compt=0
    while compt < path_length-2: # First node has two element, so we should add path_length-2
        for i in pattern_result:
            if i.split(",")[0]==path.split(",")[-1]:
                path+=","+i.split(",")[-1]
                print(compt)
                print(path)
                compt+=1
    print("path:",path)
    return path 

path = recommend()

pattern with time: [('8935,8936', 11416), ('321237,11209', 11328), ('8935,8935', 8520), ('8936,316752', 8480), ('316752,321237', 8444), ('321237,321237', 8420), ('8936,321237', 7106), ('8935,316752', 5642), ('8935,321237', 4260), ('316752,11209', 4258), ('8936,11209', 2192), ('8935,11209', 1456)]
unique pattern: ['8935,8936', '321237,11209', '8936,316752', '316752,321237']
begin_node of path: 8935,8936
0
8935,8936,316752
1
8935,8936,316752,321237
2
8935,8936,316752,321237,11209
path: 8935,8936,316752,321237,11209
8935,8936,316752,321237,11209


In [89]:
pattern = ps.topk(30, filter=lambda patt, matches: len(patt)>1)

In [130]:
a =  [('8935,8936', 2792), ('321237,11209', 2186), ('8936,321237', 1654), ('316752,11209', 1092)]
b = [x[0] for x in a]
b

['8935,8936', '321237,11209', '8936,321237', '316752,11209']

In [141]:
a = [2,]
b = [1,2]
b[1:]

[2]